## HuggingFace Summarization Transformers

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from brief_news.interface.main import get_articles

import tensorflow
import json
import requests
import pandas as pd
from itertools import chain
from transformers import pipeline

from dotenv import load_dotenv, find_dotenv
import os

/home/rriesco/.pyenv/versions/3.8.12/envs/brief-news/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-06 19:21:34.642071: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 19:21:34.642113: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TypeError: expected str, bytes or os.PathLike object, not NoneType

## Loading data

Calling the module:
  - gets links from NewsApi
  - scrapes webpages

In [ ]:
%%time
# getting actual articles from module
articles_df = get_articles('business')

In [ ]:
articles_df

In [ ]:
articles_df.article[3]

In [ ]:
articles_df.title[3]

In [ ]:
articles_df.title[4]

In [3]:
articles_df.article[4]

NameError: name 'articles_df' is not defined

## Testing API bart-large-cnn

Calling HuggingFace API

In [4]:
hf_token = os.getenv('HUGGING_API_TOKEN')

NameError: name 'os' is not defined

In [10]:
headers = {"Authorization": f"Bearer {hf_token}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

def query(payload):
    """
    Function sends post request to hugging face api for 'summarization' service
    with bart-large-cnn model
    """
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))


In [11]:
df_bart_large = articles_df.copy()
df_bart_large

,title,article,id,orig_id
0,Premarket stocks: Jay Powell's dream of the 90...,"The US economy gained 263,000 jobs in November...",0,0
1,CBI says UK recession could turn into a 'lost ...,The United Kingdom faces a “lost decade” of gr...,0,0
2,Opinion: Elon Musk's Twitter is helping to nor...,Elon Musk apparently is trying his hand at cre...,0,0
3,Foxconn dice que está restaurando la producció...,Por Juliana Liu,0,0
4,"Presentadores Amy Robach y T.J. Holmes, de “Go...",,0,0


In [12]:
%%time
# summarizing articles into 150 words with facebook/bart-large-cnn
df_bart_large['summary_text'] = df_bart_large['article'].apply(lambda article: query({'inputs':article, "parameters": {"max_length": 150}})[0]['summary_text'])
df_bart_large

CPU times: user 124 ms, sys: 17.3 ms, total: 141 ms
Wall time: 5.55 s


,title,article,id,orig_id,summary_text
0,Premarket stocks: Jay Powell's dream of the 90...,"The US economy gained 263,000 jobs in November...",0,0,"The US economy gained 263,000 jobs in November..."
1,CBI says UK recession could turn into a 'lost ...,The United Kingdom faces a “lost decade” of gr...,0,0,The Confederation of British Industry said tha...
2,Opinion: Elon Musk's Twitter is helping to nor...,Elon Musk apparently is trying his hand at cre...,0,0,Elon Musk released what he called the “Twitter...
3,Foxconn dice que está restaurando la producció...,Por Juliana Liu,0,0,Por Juliana Liu is the daughter of former Pres...
4,"Presentadores Amy Robach y T.J. Holmes, de “Go...",,0,0,CNN.com will feature iReporter photos in a wee...


In [16]:
print('Article : ')
print(df_bart_large['article'][4])
print('\n')
print(' * ' * 20)
print('\n')
print('Summary : ')
print(df_bart_large['summary_text'][4])

Article : 



 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


Summary : 
CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots. Please share your best photos of the United States with CNN iReport.


### Distilbart

In [20]:
df_distillbart = articles_df.copy()
df_distillbart

,title,article,id,orig_id
0,Premarket stocks: Jay Powell's dream of the 90...,"The US economy gained 263,000 jobs in November...",0,0
1,CBI says UK recession could turn into a 'lost ...,The United Kingdom faces a “lost decade” of gr...,0,0
2,Opinion: Elon Musk's Twitter is helping to nor...,Elon Musk apparently is trying his hand at cre...,0,0
3,Foxconn dice que está restaurando la producció...,Por Juliana Liu,0,0
4,"Presentadores Amy Robach y T.J. Holmes, de “Go...",,0,0


In [27]:
API_URL_distilbart = "https://api-inference.huggingface.co/models/philschmid/distilbart-cnn-12-6-samsum"

def query(payload):
	response = requests.post(API_URL_distilbart, headers=headers, json=payload)
	return response.json()
	


In [29]:
data = query({'inputs': df_distillbart['article'][0]})
data

[{'summary_text': "The US economy gained 263,000 jobs in November. Average hourly earnings rose by 0.55%, the fastest pace since January. The Fed's strategy to rein in inflation by raising interest rates isn't working. The dream of a sudden surge in the supply of labor is over, according to Fed Chairman Jerome Powell."}]

In [30]:
%%time
# summarizing articles into 150 words with distilbart-cnn
df_distillbart['summary_text'] = df_distillbart.article[:4].apply(lambda article: query({'inputs':article})[0]['summary_text'])
df_distillbart

CPU times: user 102 ms, sys: 11.4 ms, total: 113 ms
Wall time: 14.5 s


,title,article,id,orig_id,summary_text
0,Premarket stocks: Jay Powell's dream of the 90...,"The US economy gained 263,000 jobs in November...",0,0,"The US economy gained 263,000 jobs in November..."
1,CBI says UK recession could turn into a 'lost ...,The United Kingdom faces a “lost decade” of gr...,0,0,"The UK economy faces a ""lost decade"" of growth..."
2,Opinion: Elon Musk's Twitter is helping to nor...,Elon Musk apparently is trying his hand at cre...,0,0,Elon Musk released the “Twitter Files” from Oc...
3,Foxconn dice que está restaurando la producció...,Por Juliana Liu,0,0,Juliana Liu is a writer and photographer. She ...
4,"Presentadores Amy Robach y T.J. Holmes, de “Go...",,0,0,NaN


In [31]:
print('Bert CNN Large : ')
print(df_bart_large.summary_text[0])
print('\n')
print(' * ' * 20)
print('\n')
print('Distillbart summary : ')
print(df_distillbart.summary_text[0])

Bert CNN Large : 
The US economy gained 263,000 jobs in November, 63,000 above the consensus estimate. The larger surprise was that average hourly earnings rose by 0.55%, the fastest pace since January. The Fed’s strategy to rein in inflation by raising interest rates isn’t quite working and that more painful interest rate hikes are coming.


 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


Distillbart summary : 
The US economy gained 263,000 jobs in November. Average hourly earnings rose by 0.55%, the fastest pace since January. The Fed's strategy to rein in inflation by raising interest rates isn't working. The dream of a sudden surge in the supply of labor is over, according to Fed Chairman Jerome Powell.
